### SmiteRecommender
Takes Smite parsed data, a god, and a friend/enemy, and finds recommendations in a SmiteRecommender instance. 
Use fill() to find and store all recommendations for a given gamemode and partner relationship (friend/foe)


In [12]:
import pandas as pd
import json
import itertools
import copy
from sklearn import linear_model
import re
from tabulate import tabulate

In [13]:
class SmiteRecommender(object):
    """Represents recommendations and information for a god, given a friend or foe, and game mode.
    Requires "matches_played_by_each_god_ranked.json" -like files in SMITE_data for each gamemode you want to recommend on.
    Requires "Ranked_Conquest_SMITEdata.csv" -like files in SMITE_data for each gamemode you want to recommend on.
    Requires 'SMITE_Gods_All_Lookup.csv' and "SMITE_Items_All_Lookup.csv" (with total cost column)
    
    gamemodes:
    "ranked" (ranked conquest)
    "casual" (casual conquest)
    "joust"  (ranked joust)
    "duel"   (ranked duel)
    
    important methods:
    get_recommendations() # performs L2 regression (removed naive bayes)
        best_relics  
        best_items
    get_winrates()        # finds relative winrates and returns recommended build
        item_winrate_df 
        recommended_build
    get_relic_winrates()
        relic_winrate_df
    """
    # class variables: lookup tables for god name, item name, data and matches for each god.

    # get match data and god_matches_dict for each gamemode
    # god_matches_dict keys are (god_id + "_win) and (god_id + "_lose")
    # all data across all gamemodes will be initialized as class variables, each instance will pick a gamemode
    try: 
        with open('SMITE_data/matches_played_by_each_god_ranked.json', 'r') as _f:
            _ranked_god_matches_dict = json.load(_f)
        _ranked_data = pd.read_csv('SMITE_data/Ranked_Conquest_SMITEdata.csv', index_col='god_id')
    except FileNotFoundError:
        _ranked_god_matches_dict = None
        _ranked_data = None
    try:     
        with open('SMITE_data/matches_played_by_each_god_casual.json', 'r') as _f:
            _casual_god_matches_dict = json.load(_f)
        _casual_data = pd.read_csv('SMITE_data/Casual_Conquest_SMITEdata.csv', index_col='god_id')
    except FileNotFoundError:
        _casual_god_matches_dict = None
        _casual_data = None
    try: 
        with open('SMITE_data/matches_played_by_each_god_duel.json', 'r') as _f:
            _duel_god_matches_dict = json.load(_f)
        _duel_data = pd.read_csv('SMITE_data/Ranked_Duel_SMITEdata.csv', index_col='god_id')
    except FileNotFoundError:
        _duel_god_matches_dict = None
        _duel_data = None
    try: 
        with open('SMITE_data/matches_played_by_each_god_joust.json', 'r') as _f:
            _joust_god_matches_dict = json.load(_f)
        _joust_data = pd.read_csv('SMITE_data/Ranked_Joust_SMITEdata.csv', index_col='god_id')
    except FileNotFoundError:
        _joust_god_matches_dict = None
        _joust_data = None
            
    _gods_lookup = pd.read_csv("SMITE_data/SMITE_Gods_All_Lookup.csv").set_index('id')
    _item_lookup = pd.read_csv("SMITE_data/SMITE_Items_All_Lookup.csv").set_index('ItemId')
    
    _all_relics_idx = set(_item_lookup.index[_item_lookup.Type=='Active'])
    _all_items_idx = set(_item_lookup.index[_item_lookup.ItemTier==3]).difference(_all_relics_idx)
    # include 'starter' items (they are relevant for guardians)
    _all_items_idx.update({11122, 11123, 9089, 8247, 8268, 8538, 8987})
    _all_gods_idx = set(_gods_lookup.index)
    
    _num_relics = len(_all_relics_idx)
    _num_items = len(_all_items_idx)
    _num_gods = len(_all_gods_idx)
    
    def find_god(god_):
        """Returns god name if given id (int). Returns id if given exact god name (str). Must be properly capitalized!"""
        if isinstance(god_, str):
            return(SmiteRecommender._gods_lookup[SmiteRecommender._gods_lookup["Name"]==god_].index.item())
        elif isinstance(god_, int):
            return(SmiteRecommender._gods_lookup.loc[god_, 'Name'])
        
    def find_item(item_):
        """Returns item name if given id (int). Returns id if given exact item name (str)."""
        if isinstance(item_, str):
            return(SmiteRecommender._item_lookup[SmiteRecommender._item_lookup["DeviceName"]==item_].index.item())
        elif isinstance(item_, int):
            return(SmiteRecommender._item_lookup.loc[item_, 'DeviceName'])
                   
    # It's a big one
    def __init__(self, god_id, partner_id, friend=False, gamemode ="ranked", enemy_can_dup_character=True):
        """TODO
        :params god_id, name, pantheon, role, title: descriptive info about self
        :params friend_ or foe_ :                    descriptive info about partner
        :param info, info_table:                     Short summary
        :param num_matches, winrate, gamemode:       number of matches together, winrate vs/w partner, gamemode
        :param _df:                                   dataframe of match info, to be used by get_recommendations
        """
        if god_id==partner_id and friend==True:
            print("Error: cannot have multiples of the same god on your team")
            return
        if god_id==partner_id and friend==False and enemy_can_dup_character==False:
            print("Error: cannot have the same god on the enemy team")
            return
        if friend==True and gamemode=="duel":
            print("Error: cannot have teammate in 1v1")
            return
        
        self.god_id = god_id
        self._name = SmiteRecommender._gods_lookup.Name[god_id]
        self._pantheon = SmiteRecommender._gods_lookup.Pantheon[god_id]
        self._role = SmiteRecommender._gods_lookup.Roles[god_id]
        self._title = SmiteRecommender._gods_lookup.Title[god_id]
        self.gamemode = gamemode
        
        if gamemode == "ranked":
            god_matches_dict = SmiteRecommender._ranked_god_matches_dict
            data = SmiteRecommender._ranked_data
        elif gamemode == "casual":
            god_matches_dict = SmiteRecommender._casual_god_matches_dict
            data = SmiteRecommender._casual_data
        elif gamemode == "joust":
            god_matches_dict = SmiteRecommender._joust_god_matches_dict
            data = SmiteRecommender._joust_data
        elif gamemode == "duel":
            god_matches_dict = SmiteRecommender._duel_god_matches_dict
            data = SmiteRecommender._duel_data
        else:
            return("ERROR: gamemode not valid. Choose ranked, casual, joust, or duel.")
        if god_matches_dict is None or data is None:
            print("ERROR: No data found for given gamemode.")
            return
        
######### Get only relevant matches (intersection of matches played by each character, given their relationship)
        if friend:
            matches_won_together = set(god_matches_dict[str(god_id) + "_win"]) \
                     .intersection(set(god_matches_dict[str(partner_id) + "_win"]))
            matches_lost_together = set(god_matches_dict[str(god_id) + "_lose"]) \
                     .intersection(set(god_matches_dict[str(partner_id) + "_lose"]))
            self._matches = matches_won_together.union(matches_lost_together)
        elif not friend:
            matches_won_against = set(god_matches_dict[str(god_id) + "_win"]) \
                    .intersection(set(god_matches_dict[str(partner_id) + "_lose"]))
            matches_lost_against = set(god_matches_dict[str(god_id) + "_lose"]) \
                     .intersection(set(god_matches_dict[str(partner_id) + "_win"]))
            self._matches = matches_won_against.union(matches_lost_against)
        self.num_matches = len(self._matches)
        
        if friend:
            self._friend_god_id = partner_id
            self._friend_name = SmiteRecommender._gods_lookup.Name[partner_id]
            self._friend_pantheon = SmiteRecommender._gods_lookup.Pantheon[partner_id]
            self._friend_role = SmiteRecommender._gods_lookup.Roles[partner_id]
            self._friend_title = SmiteRecommender._gods_lookup.Title[partner_id]
            self.info = "{} w/ {} | {} {} matches".format(self._name, self._friend_name, self.num_matches, gamemode)
        elif not friend:
            self._foe_god_id = partner_id
            self._foe_name = SmiteRecommender._gods_lookup.Name[partner_id]
            self._foe_pantheon = SmiteRecommender._gods_lookup.Pantheon[partner_id]
            self._foe_role = SmiteRecommender._gods_lookup.Roles[partner_id]
            self._foe_title = SmiteRecommender._gods_lookup.Title[partner_id]
            self.info = "{} vs {} | {} {} matches".format(self._name, self._foe_name, self.num_matches, gamemode)        
        
        min_num_matches = 100
        if self.num_matches < min_num_matches:
            print("Only {} matches.\nNot enough for recommendations.".format(self.num_matches))
            return  
######### Construct df of all matches between self and partner #######
        # First: make names for all binary feature variables
        gods_minus_self = SmiteRecommender._all_gods_idx.difference({str(god_id)})
        self_relic_names = {"relic" + str(relic_id) for relic_id in SmiteRecommender._all_relics_idx}
        self_relic_names.update({"relic0"}) #for no relic purchased
        self_item_names = {"item" + str(item_id) for item_id in SmiteRecommender._all_items_idx}
        self_item_names.update({"item0"}) # for no item purchased
        friend_feature_names = {"friend" + str(friend_id) for friend_id in gods_minus_self}  
        if enemy_can_dup_character:
            foe_feature_names = {"foe" + str(foe_id) for foe_id in SmiteRecommender._all_gods_idx}
        else:
            foe_feature_names = {"foe" + str(foe_id) for foe_id in gods_minus_self}
        # remove partner
        if friend:
            friend_feature_names.difference_update({"friend" + str(partner_id)})
        else:
            foe_feature_names.difference_update({"foe" + str(partner_id)})
        feature_names = itertools.chain(self_relic_names, self_item_names, friend_feature_names, foe_feature_names)

        # Second: initialize empty dict (to eventually become rows of dataframe)
        features = {}
        for name in feature_names:
            features[name] = None 

        # Third: make row_dict for each match, fill, and append to matches_list.
        def get_row_info(match, matchdf, row_dict, self_win):
            # fill in dict values for all wanted information given a df with only info for 1 match (10 rows in conquest)
            # updates row_dict without returning anything.
            for god, row in matchdf.iterrows():
                if row["win"] == self_win:
                    if god == god_id:
                        row_dict.update({"win" :self_win,
                                         "match_id" : int(match),
                                         "gold_per_minute" : row["Gold_Per_Minute"],
# adds 2ms                               "avg_gold_per_minute_friends" : matchdf.loc[matchdf.win==self_win].Gold_Per_Minute.sum()/5,
#                                        "avg_gold_per_minute_foes" : matchdf.loc[matchdf.win!=self_win].Gold_Per_Minute.sum()/5,
                                         "relic" + str(row["relic0"]) : 1,
                                         "relic" + str(row["relic1"]) : 1,
                                         "item" + str(row["item0"]) : 1,
                                         "item" + str(row["item1"]) : 1,
                                         "item" + str(row["item2"]) : 1,
                                         "item" + str(row["item3"]) : 1,
                                         "item" + str(row["item4"]) : 1,
                                         "item" + str(row["item5"]) : 1,
                                        })
                    elif not friend:
                        row_dict["friend" + str(god)] = 1
                    elif friend and god != partner_id:
                        row_dict["friend" + str(god)] = 1
                    elif friend and god == partner_id:
                        row_dict['gold_per_minute_partner'] = row["Gold_Per_Minute"]
                
                if row["win"] != self_win:
                    if friend:
                        row_dict["foe" + str(god)] = 1
                    elif not friend and god != partner_id:
                        row_dict["foe" + str(god)] = 1
                    elif not friend and god == partner_id:
                        row_dict['gold_per_minute_partner'] = row["Gold_Per_Minute"]
                        
        # Still Third: fill and append all matches. Handles duplicates as well, including mirror matches (hopefully)
        matches_list = []
        for match in self._matches:              # 7 ms each match                
            row_dict = copy.deepcopy(features)   # %%timeit   556 µs to copy
            dup = False
            matchdf = data.loc[data.match_id==int(match)] # make small df for each match.
            # if they never bought a (free) relic at level 1, they must have dropped immediately. Skip those games.
            if any(matchdf['relic0'] == 0):
                continue
                
            self_win = matchdf.get_value(god_id,"win") # will return an array if there are duplicates of god in match
            # Check if there are more than 1 of this god in this match, i.e. if enemy_can_dup_character
            if enemy_can_dup_character:
                if self_win.size>1:
                    self_win = self_win[0] #self_win refers to the first time the god shows up
                    dup = True
            
            get_row_info(match, matchdf, row_dict, self_win)
            matches_list.append(row_dict)
            
            if dup:
                row_dict_extra = copy.deepcopy(features)
                opp_win = not self_win # this allows us to fill the row from the perspective of the duplicate god
                get_row_info(match, matchdf, row_dict_extra, opp_win)
                row_dict_extra.update({"match_id" : 1000*int(match)}) # simple way to tag this as an extra row
                matches_list.append(row_dict_extra)
                
        # Fourth: Make dataframe
        if len(matches_list) < min_num_matches:
            print("Only {} matches!\nNot enough for recommendations".format(len(matches_list)))
            return
        self._df = pd.DataFrame(matches_list).set_index('match_id')
        self._df.dropna(subset=["win"], inplace=True) # Check for and remove possible problem rows
        self.num_matches = len(self._df)  # update num_matches (after throwing out bad matches)
        
        # Finally: get winrate and info_table
        self.winrate = pd.to_numeric(self._df['win']).sum()/self.num_matches
        if friend:
            self.info_table = tabulate([[self._name + ': ' + self._title, self._pantheon, self._role, "w/", "{:3.1%}".format(self.winrate)],\
            [self._friend_name + ': ' + self._friend_title, self._friend_pantheon, self._friend_role, str(self.num_matches) +" matches", ""]],\
            headers=['God', 'Pantheon', "Role", gamemode], tablefmt='fancy_grid')
        elif not friend:
            self.info_table = tabulate([[self._name + ': ' + self._title, self._pantheon, self._role, "vs", "{:3.1%}".format(self.winrate)],\
            [self._foe_name + ': ' + self._foe_title, self._foe_pantheon, self._foe_role, str(self.num_matches) +" matches", ""]],\
            headers=['God', 'Pantheon', "Role", gamemode, "Win Rate"], tablefmt='fancy_grid')
        #__init__ complete
        
    def __get_best_relics(self, relics_coefs, relic_frequencies):
        """ Creates self.best_relics df with info about relic choices, most popular and most recommended, with ratings and winrates
        Note: chosing arbitrary cutoff point of 3% relic use
        # this may allow some lucky outliers into the prediction, which is less of a big deal for relic recommendations
        """ 
        percent_threshold = .03
        min_cutoff = (self.num_matches)*percent_threshold             
        n=0
        most_rec=[]
        coefs = []
        winrates = []
        winrates_pop = []
        most_pop = relic_frequencies.index.values[:2]
        
        # fill most_rec, coefs, and winrates
        for relicnum in relics_coefs.index:
            if n<2 and relic_frequencies[relicnum]>min_cutoff:
                most_rec.append(relicnum)
                coefs.append(relics_coefs[relicnum]*100)
                wr_info = self._df.loc[self._df[relicnum]==1, "win"].value_counts()
                total = wr_info.sum()
                try:
                    winrates.append(wr_info[True]/total)
                except (KeyError, IndexError):
                    winrates.append(0.)
                    print(total, "matches with relic. Error raised finding winrate")
                n+=1
            elif n==2: break
        # fill winrates_pop
        for relicnum in most_pop:
            wr_info = self._df.loc[self._df[relicnum]==1, "win"].value_counts()
            total = wr_info.sum()
            try:
                winrates_pop.append(wr_info[True]/total)
            except (KeyError, IndexError):
                winrates_pop.append(0.)
        # fill paired winrates
        paired_wr_info = self._df.loc[(self._df[most_rec[0]]==1) & (self._df[most_rec[1]]==1) , "win"].value_counts()
        paired_pop_wr_info = self._df.loc[(self._df[most_pop[0]]==1) & (self._df[most_pop[1]]==1) , "win"].value_counts()
        paired_L2_total = paired_wr_info.sum()
        paired_pop_total = paired_pop_wr_info.sum()
        try:
            paired_wr = paired_wr_info[True]/paired_L2_total
        except (KeyError, IndexError):
            paired_wr = 0.
            print(paired_L2_total, "matches with pair of relics. Error raised finding winrate")
        try:
            paired_pop_wr = paired_pop_wr_info[True]/paired_pop_total
        except (KeyError, IndexError):
            paired_pop_wr = 0.
            print(paired_pop_total, "matches with pair of relics. Error raised finding winrate")                

        self.best_relics = pd.DataFrame({"Relics_most_pop" : 
                                    [SmiteRecommender.find_item(int(x.replace("relic", ""))) for x in most_pop],
                                    'Relics_most_pop_L2' : relics_coefs[most_pop].values*100,
                                    'Relics_most_pop_winrate' : winrates_pop,
                                    'Relics_most_pop_wr_paired' :   [paired_wr, ""],
                                    'Relics_L2' : 
                                    [SmiteRecommender.find_item(int(x.replace("relic", ""))) for x in most_rec],
                                    'Relics_L2_coef' : coefs,
                                    'Relics_L2_winrate' : winrates,
                                    'Relics_L2_wr_paired' :  [paired_pop_wr, ""]
                                   }, columns = ['Relics_most_pop', 'Relics_most_pop_L2', "Relics_most_pop_winrate",
                                                'Relics_most_pop_wr_paired', 'Relics_L2', 'Relics_L2_coef',
                                                'Relics_L2_winrate','Relics_L2_wr_paired'])

        
    def __get_best_items(self, items_coefs, gold_df, item_frequencies):
        """ Creates self.best_items df with info about item choices, most popular, most recommended, 
        and Gold Adjusted most recommended, with ratings and winrates
        Note: chosing arbitrary cutoff point. Item must be used by god in >5% of matches
        NOTE: This could have a large effect on recommendations. We don't want lucky outliers jumping to the top.
        Also creates self._most_pop_starter_item (based purely on popularity) for later use in recommendation df
        """ 
        percent_threshold = .05
        num_items_full_build = 6
        # Want to suggest starter items seperately (they are so cheap it throws off the cost normalization, and are meant for early game)
        starter_items = ({"item11122", 'item11123', 'item9089', 'item8247', 'item8268', 'item8538', 'item8987'})     
        best_starters = (itemnum for itemnum in item_frequencies.index if itemnum in starter_items)                     
        self._most_pop_starter_item = SmiteRecommender.find_item(int(next(best_starters).replace("item", "")))    
        min_cutoff = self.num_matches*percent_threshold
        n=0
        most_rec=[]
        coefs = []
        winrates = []
        winrates_pop = []
        gold_most_rec = []
        gold_coefs = []
        gold_winrates = []
        most_pop = item_frequencies.index.values[:6]
        
        # fill most_rec, coefs, and winrates
        for itemnum in items_coefs.index:
            if n<num_items_full_build and item_frequencies[itemnum]>min_cutoff and itemnum not in starter_items and\
            int(itemnum.replace("item", "")) in SmiteRecommender._all_items_idx: # screen out tier 2 items
                most_rec.append(itemnum)
                coefs.append(items_coefs[itemnum]*100)
                wr_info = self._df.loc[self._df[itemnum]==1, "win"].value_counts()
                total = wr_info.sum()
                try:
                    winrates.append(wr_info[True]/total)
                except (KeyError, IndexError):
                    winrates.append(0.)
                    print(total, "matches with item. Error raised finding winrate")
                n+=1
            elif n==num_items_full_build: break
        # fill winrates_pop
        for itemnum in most_pop:
            wr_info = self._df.loc[self._df[itemnum]==1, "win"].value_counts()
            total = wr_info.sum()
            try:
                winrates_pop.append(wr_info[True]/total)
            except (KeyError, IndexError):
                winrates_pop.append(0.)
        # fill gold_adjusted variables
        n=0
        for itemnum in gold_df.index:
            if n<num_items_full_build and item_frequencies[itemnum]>min_cutoff and itemnum not in starter_items and\
            int(itemnum.replace("item", "")) in SmiteRecommender._all_items_idx: # screen out tier 2 items
                gold_most_rec.append(itemnum)
                gold_coefs.append(gold_df.loc[itemnum, "coef/cost"])          
                wr_info = self._df.loc[self._df[itemnum]==1, "win"].value_counts()
                total = wr_info.sum()
                try:
                    gold_winrates.append(10000*wr_info[True]/(total*gold_df.loc[itemnum, "cost"]))                                        
                except (KeyError, IndexError):
                    gold_winrates.append(0.)
                n+=1   
            if n==num_items_full_build:
                break        
        
        self.best_items = pd.DataFrame({"Items_most_pop" :
                                   [SmiteRecommender.find_item(int(x.replace("item", ""))) for x in most_pop],
                                   "Items_most_pop_L2" : items_coefs[most_pop].values*100,
                                   'Items_most_pop_winrate' : winrates_pop,
                                   'Items_L2' : 
                                    [SmiteRecommender.find_item(int(x.replace("item", ""))) for x in most_rec],
                                   'Items_L2_coef' : coefs,
                                   'Items_L2_winrate' : winrates,
                                   'Items_gold_adjusted_L2' :
                                   [SmiteRecommender.find_item(int(x.replace("item", ""))) for x in gold_most_rec],
                                   'Items_gold_adjusted_L2_coef' : gold_coefs,
                                   "Items_gold_adjusted_L2_winrate" : gold_winrates
                                  }, columns = ['Items_most_pop', 'Items_most_pop_L2', "Items_most_pop_winrate", 'Items_L2',
                                                'Items_L2_coef', 'Items_L2_winrate', 'Items_gold_adjusted_L2',
                                                'Items_gold_adjusted_L2_coef','Items_gold_adjusted_L2_winrate']) 
    
    
    def __make_gold_df(self, items_coefs):
        # returns df with columns L2_coef/cost and cost,sorted by coef/cost
        # assigns a coef of -1 to items with cost of 0 or less 
        # for now All tier 2 items are explicitely at a cost of -1 in _item_lookup
        # coef/cost mulitplied by arbitrary factor 100000 for to look nicer
        coefs_dict = {}
        cost_dict = {}
        for item in items_coefs.index:
            cost = SmiteRecommender._item_lookup[SmiteRecommender._item_lookup.index==int(item.replace("item", ""))].TotalCost.item()
            if cost>0:
                coefs_dict[item]=10000*items_coefs[item]/cost
            else:
                coefs_dict[item]= -1
            cost_dict[item] = cost
        return(pd.DataFrame([coefs_dict, cost_dict], index=["coef/cost","cost"]).T.sort_values(by="coef/cost",ascending=False))
    
    def __get_rec(self):
        """Creates self.recommendations, a df of recommended items based on best_items output
        with winrate and wr/cost info. Includes starter items, and relics. 
        Boots will always be the first recommendation (and there will always only be one!)
        """
        # make recommendations based on replacing the 2 worst performing items in the top five, with the 2 items with the
        # highest l2 coeff/cost value (that are not already in the the top five)
        boots = {"Warrior Tabi", "Ninja Tabi", "Reinforced Greaves", "Talaria Boots", 
                 "Shoes of the Magi", "Shoes of Focus", "Reinforced Shoes", "Travelers Shoes"}
        starter_items = ({"Watcher's Gift", "Bumba's Mask", 'Mark of the Vanguard', "Death's Toll", "Vampiric Shroud", 'Bluestone Pendant', 'Soul Stone'}) 
    

        recommended_build = [self._most_pop_starter_item]
        if len(recommended_build)==0:
            recommended_build=[""]
        build_winrate = [""]
        build_wr_per_cost = [""]
        
        num_items = len(self.best_items)
        most_pop_sorted = pd.Series(data=self.best_items.Items_most_pop.values, index=self.best_items.Items_most_pop_L2)
        most_pop_sorted.sort_index(ascending=False, inplace=True) 
        
        # find best boots first. look in gold adjusted coefs, if no boots there, look in most popular.
        for i in range(num_items):
            item = self.best_items.loc[i,"Items_gold_adjusted_L2"]
            if item in boots:
                recommended_build.append(item)
                build_winrate.append(self.best_items.loc[i,"Items_L2_winrate"])
                build_wr_per_cost.append(self.best_items.loc[i,"Items_gold_adjusted_L2_winrate"])
                break
        else:
            for item in most_pop_sorted:
                if item in boots:
                    recommended_build.append(item)
                    build_winrate.append(self.best_items.loc[self.best_items.Items_most_pop==item,"Items_most_pop_winrate"].item())
                    cost = SmiteRecommender._item_lookup[SmiteRecommender._item_lookup.DeviceName==item].TotalCost.item()
                    build_wr_per_cost.append(10000*self.best_items.loc[i,"Items_most_pop_winrate"]/cost)
                    break
        # Now fill with three best (non-boots) items from most popular
        for item in most_pop_sorted:
            if item not in boots:
                recommended_build.append(item)
                build_winrate.append(self.best_items.loc[self.best_items.Items_most_pop==item,"Items_most_pop_winrate"].item())
                cost = SmiteRecommender._item_lookup[SmiteRecommender._item_lookup.DeviceName==item].TotalCost.item()
                build_wr_per_cost.append(10000*self.best_items.loc[i,"Items_most_pop_winrate"]/cost)
            if len(recommended_build) == 5: break
        # Now fill with best two (non-boots) items from most recommended (that aren't already chosen)
        for i in range(num_items):
            item = self.best_items.loc[i,"Items_gold_adjusted_L2"]
            if item not in boots and item not in recommended_build:
                recommended_build.append(item)
                build_winrate.append(self.best_items.loc[i,"Items_L2_winrate"])
                build_wr_per_cost.append(self.best_items.loc[i,"Items_gold_adjusted_L2_winrate"])
            if len(recommended_build) == 7: break
        else: # if build still not full, add another popular item
            for item in most_pop_sorted:
                if item not in boots and item not in recommended_build:
                    recommended_build.append(item)
                    build_winrate.append(self.best_items.loc[self.best_items.Items_most_pop==item,"Items_most_pop_winrate"].item())
                    cost = SmiteRecommender._item_lookup[SmiteRecommender._item_lookup.DeviceName==item].TotalCost.item()
                    build_wr_per_cost.append(10000*self.best_items.loc[i,"Items_most_pop_winrate"]/cost)
                if len(recommended_build) == 7: break
        # in case of problems, fill in with empty values
        while(len(recommended_build) < 7):
            recommended_build.append("")
            build_winrate.append(0.)
            build_wr_per_cost.append(0.)
            
        # add relics
        recommended_build.append(self.best_relics["Relics_L2"][0])
        build_winrate.append(self.best_relics["Relics_L2_winrate"][0])
        build_wr_per_cost.append("")
        recommended_build.append(self.best_relics["Relics_L2"][1])
        build_winrate.append(self.best_relics["Relics_L2_winrate"][1])
        build_wr_per_cost.append("")
        
        # in case of problems, fill in with empty values
        while(len(recommended_build) < 9):
            recommended_build.append("")
            build_winrate.append(0.)
            build_wr_per_cost.append(0.)
        
        self.recommendation = pd.DataFrame({"recommendations" : recommended_build,
                                             "win_rate" : build_winrate,
                                             "win_rate_per_cost" : build_wr_per_cost
                                            }, index = ["Starter Item", "Boots", "Item2", "Item3",
                                                       "Item4", "Item5", "Item6", "Relic1", "Relic2"])
    def get_recommendations(self):
        """ Performs L2 regularized Logistic Regression on win outcome and produces recommendations.
        Creates instance variables:
        self.best_relics  -pandas df of 2 most frequent and recommended relics
        self.best_items   -pandas df of 5 most frequent and recommended items,
                            includes logodds/cost of item as a (gold) normalized prediction, and winrates
        """       
        """
        def get_naive_bayes(self): 
            items = [col for col in list(self._df) if col.startswith('item')]
            item_wins = self._df.loc[~self._df['win'], items].sum()
            item_losses = self._df.loc[self._df['win'], items].sum()
            nwins = self.winrate*self.num_matches
            nloss = self.num_matches-nwins
            loss_odds = nloss/nwins
            # loss_odds is our prior.
            # vectorized
            iw = (item_wins/nwins).fillna(.00001)
            inw = (item_losses/nloss).fillna(.00001)
            naive_bayes = 1/(1+(loss_odds*(inw/iw)))
            return(naive_bayes.sort_values(ascending=False))
        """ 
        # drop unhelpful(?) features (keeping item0 and relic0), replace naN with 0s
        # Special case for duel 1v1
        if self.gamemode == "duel":
            features = self._df.select(lambda x: not re.search('foe\d+', x) and not re.search('friend\d+', x) , axis=1)
            features = features.drop(["win", 'gold_per_minute','gold_per_minute_partner'], axis = 1).fillna(0)
        else:
            features = self._df.drop(["win",'gold_per_minute','gold_per_minute_partner'], axis = 1).fillna(0)
        target = self._df.win.fillna(0)

        item_frequencies = features.filter(regex=(r"item\d")).sum().sort_values(ascending=False)
        relic_frequencies = features.filter(regex=(r"relic\d")).sum().sort_values(ascending=False)
        item_frequencies.drop("item0", inplace=True)

        model = linear_model.LogisticRegressionCV(Cs=100, fit_intercept=True, cv=3,
                                                  penalty='l2', solver='liblinear',  verbose=0) #n_jobs=-1,
        try:
            model.fit(features,target)
        except ValueError:
            print("Model Failed")
            return
        print('CV logistic ridge regression performed. Lambda chosen:', 1/model.C_)
        coefs =  pd.Series(dict(zip(features.columns,model.coef_[0].T)))
        items_coefs= coefs.filter(regex=(r"item\d")).sort_values(ascending=False)
        items_coefs.drop("item0", inplace=True)
        relics_coefs = coefs.filter(regex=(r"relic\d")).sort_values(ascending=False)
        relics_coefs.drop("relic0", inplace=True)
        
        gold_df = self.__make_gold_df(items_coefs)     
        self.__get_best_relics(relics_coefs, relic_frequencies)
        self.__get_best_items(items_coefs, gold_df, item_frequencies)
        self.__get_rec()

In [ ]:
# Hunters (Physical, ADC):
# Ah Muzen Cab  1956
# Neith  1872
# Rama   2002

# Warriors (Physical):
# Odin   1669
# Vamana 1872

# Guardians (magical, Support):
# Sobek  1747
# Ymir   1670
# Athena 1919
# Ares 1782

# Mages (magical):
# Vulcan 1869
# Ra  1698
# Zeus 1672
# Janus 1999

# Assassin(physical, jungler):
# Camazotz  2189   (newest god (1 month old))
# Loki  1797
# Thor  1779

#SmiteRecommender.find_god("Anhur")

In [ ]:
#1956-2056

In [ ]:
def fill(gamemode, friend):
    if friend:
        folder = gamemode + "_friend/"
    elif not friend:
        folder = gamemode + "_foe/"
    path = "SMITE_recommendations/" + folder
    n=0
    for god1 in SmiteRecommender._all_gods_idx:
        n+=1
        print("\n",n,"Starting", SmiteRecommender.find_god(int(god1)), "\n")
        for god2 in SmiteRecommender._all_gods_idx:
            smite_rec = SmiteRecommender(int(god1), int(god2), gamemode=gamemode, friend=friend)
            try:
                with open(path + "{}-{}_info.txt".format(int(god1), int(god2)), "wb") as f:
                    f.write(smite_rec.info_table.encode("utf-8") ) #byte encoded
            except: pass
            try:
                smite_rec.get_recommendations()
                smite_rec.best_items.to_csv(path + "{}-{}_items.csv".format(int(god1), int(god2)), index=False)
                smite_rec.best_relics.to_csv(path + "{}-{}_relics.csv".format(int(god1), int(god2)), index=False)
                smite_rec.recommendation.to_csv(path + "{}-{}_recommendation.csv".format(int(god1), int(god2)))
            except AttributeError: pass

In [ ]:
if __name__ == "__main__":
    # gamemodes: "ranked", "joust", "casual", "duel"
    fill(gamemode="joust", friend=False)